<h1>Battle of Neighborhoods</h1>
<h2>Edward Muñoz Garro</h2>
<hr/>

<h3>Introduction / Business problem</h3>

Every day a large number of tourists visit various places, however as a tourist it is not always easy to find places of interest or even those that represent an immediate need such as hotels and restaurants. This project tries to show the sites of interest closest to a marked location on the map.

The following problems will be solved in this assignment::
<ol>
    <li>Find all the restaurants within the city.</li>
    <li>Identify the mean of these places.</li>
    <li>Locate the hotels near the coordinates of the mean.</li>
    <li>Locating the closest hotel is the result of the project.</li>
</ol>

<h3>Description of the data</h3>

I used Foursquare to locate the venues and used their data.
after defining their means I used Foursquare again to identify the nearby hotels and used that as the second dataset.

In [1]:
# Warning: this could take several minutes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [4]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

<h3>Prepare Foursquare credentials</h3>

In [5]:
CLIENT_ID = 'N3PL5V2SQ1W1MRVSNEUQJYSMF2BZAA42MYMRXEI31ZJTZ2FM' # your Foursquare ID
CLIENT_SECRET = 'DOAAAWPMVBIBWHSFET5WNRNWMNHJKEBWUB1J35DGPDE0L0X5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N3PL5V2SQ1W1MRVSNEUQJYSMF2BZAA42MYMRXEI31ZJTZ2FM
CLIENT_SECRET:DOAAAWPMVBIBWHSFET5WNRNWMNHJKEBWUB1J35DGPDE0L0X5


<h3>Explore Budapest</h3>

In [6]:
request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "restaurant",
    "near": "Budapest",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [7]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [8]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Budapest', 'Budapest')

In [9]:
latitude = 47.497913
longitude = 19.040236
radius = 10000
LIMIT = 1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [10]:
search_query = 'american restauant'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1b7f6740ea3c7ec7f3f70a'},
 'response': {'venues': [{'id': '571904af498eae8cbe0ba0df',
    'name': 'American Corner Budapest',
    'location': {'address': 'Fővám tér 13-15.',
     'lat': 47.485341,
     'lng': 19.060187,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.485341,
       'lng': 19.060187}],
     'distance': 2051,
     'postalCode': '1093',
     'cc': 'HU',
     'neighborhood': 'Ferencváros',
     'city': 'Budapest',
     'state': 'Budapest',
     'country': 'Magyarország',
     'formattedAddress': ['Budapest',
      'Fővám tér 13-15.',
      '1093',
      'Magyarország']},
    'categories': [{'id': '4bf58dd8d48988d1a7941735',
      'name': 'College Library',
      'pluralName': 'College Libraries',
      'shortName': 'Library',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/library_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595638202',
    'hasPerk': False},
   {'

In [12]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1a7941735', 'name': 'C...",False,571904af498eae8cbe0ba0df,Fővám tér 13-15.,HU,Budapest,Magyarország,NaN,2051,"[Budapest, Fővám tér 13-15., 1093, Magyarország]","[{'label': 'display', 'lat': 47.485341, 'lng':...",47.485341,19.060187,Ferencváros,1093,Budapest,American Corner Budapest,v-1595638202,NaN
1,"[{'id': '4bf58dd8d48988d110951735', 'name': 'S...",False,4fc647c7e4b0bf269c3a47a7,NaN,HU,NaN,Magyarország,NaN,1018,[Magyarország],"[{'label': 'display', 'lat': 47.50572863867136...",47.505729,19.047282,NaN,NaN,NaN,American Crew Salon&Shop,v-1595638202,NaN
2,"[{'id': '4bf58dd8d48988d1f6931735', 'name': 'G...",False,4dc808b8d22dafda2fe1e10a,József Attila u. 18.,HU,Budapest,Magyarország,NaN,1128,"[Budapest, József Attila u. 18., Magyarország]","[{'label': 'display', 'lat': 47.49964595552910...",47.499646,19.055020,NaN,NaN,Budapest,American Service Travel,v-1595638202,NaN
3,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",False,5ae0a7888496ca0c1dfe4cff,Bajcsi-Zsilinszky út 70.,HU,Budapest,Magyarország,NaN,1564,"[Budapest, Bajcsi-Zsilinszky út 70., 1054, Mag...","[{'label': 'display', 'lat': 47.50791973772882...",47.507920,19.054836,NaN,1054,Budapest,I55 American Bar & Restaurant,v-1595638202,NaN
4,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",False,580fabe938fadc332905d27f,NaN,HU,Budapest,Magyarország,NaN,1611,"[Budapest, 1024, Magyarország]","[{'label': 'display', 'lat': 47.511208, 'lng':...",47.511208,19.031746,NaN,1024,Budapest,American Burger,v-1595638202,NaN


In [13]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,American Corner Budapest,College Library,Fővám tér 13-15.,HU,Budapest,Magyarország,NaN,2051,"[Budapest, Fővám tér 13-15., 1093, Magyarország]","[{'label': 'display', 'lat': 47.485341, 'lng':...",47.485341,19.060187,Ferencváros,1093,Budapest,571904af498eae8cbe0ba0df
1,American Crew Salon&Shop,Salon / Barbershop,NaN,HU,NaN,Magyarország,NaN,1018,[Magyarország],"[{'label': 'display', 'lat': 47.50572863867136...",47.505729,19.047282,NaN,NaN,NaN,4fc647c7e4b0bf269c3a47a7
2,American Service Travel,General Travel,József Attila u. 18.,HU,Budapest,Magyarország,NaN,1128,"[Budapest, József Attila u. 18., Magyarország]","[{'label': 'display', 'lat': 47.49964595552910...",47.499646,19.055020,NaN,NaN,Budapest,4dc808b8d22dafda2fe1e10a
3,I55 American Bar & Restaurant,New American Restaurant,Bajcsi-Zsilinszky út 70.,HU,Budapest,Magyarország,NaN,1564,"[Budapest, Bajcsi-Zsilinszky út 70., 1054, Mag...","[{'label': 'display', 'lat': 47.50791973772882...",47.507920,19.054836,NaN,1054,Budapest,5ae0a7888496ca0c1dfe4cff
4,American Burger,Burger Joint,NaN,HU,Budapest,Magyarország,NaN,1611,"[Budapest, 1024, Magyarország]","[{'label': 'display', 'lat': 47.511208, 'lng':...",47.511208,19.031746,NaN,1024,Budapest,580fabe938fadc332905d27f
5,American Airlines,Office,NaN,HU,NaN,Magyarország,NaN,2579,[Magyarország],"[{'label': 'display', 'lat': 47.52101070265701...",47.521011,19.037438,NaN,NaN,NaN,4e65d45252b1260c148dcb08
6,American Crew Shop,Salon / Barbershop,Lajos Utca 6.,HU,Budapest,Magyarország,NaN,2772,"[Budapest, Lajos Utca 6., Magyarország]","[{'label': 'display', 'lat': 47.52278240630487...",47.522782,19.038188,NaN,NaN,Budapest,4ee792f70e0175b50b0e962b
7,American Express Customer Service,Bank,Váci út 76.,HU,Budapest,Magyarország,NaN,3459,"[Budapest, Váci út 76., 1133, Magyarország]","[{'label': 'display', 'lat': 47.524967, 'lng':...",47.524967,19.062885,NaN,1133,Budapest,5cad9852b9ac380039b978fc
8,American Bbq Picniq,Street Food Gathering,Hengermalom út 51.,HU,Budapest,Magyarország,NaN,4425,"[Budapest, Hengermalom út 51., 1117, Magyarors...","[{'label': 'display', 'lat': 47.45984, 'lng': ...",47.459840,19.057175,NaN,1117,Budapest,598df07a826444188d622536
9,AC Seafood American And Vietnamese Streetfood,American Restaurant,Akadémia utca 11.,HU,Budapest,Magyarország,Széchenyi utca,729,"[Budapest, Akadémia utca 11. (Széchenyi utca),...","[{'label': 'display', 'lat': 47.50289892302387...",47.502899,19.046533,NaN,1054,Budapest,54df8979498e9d4d76f81982


In [14]:
dataframe_filtered.name

0                              American Corner Budapest
1                              American Crew Salon&Shop
2                               American Service Travel
3                         I55 American Bar & Restaurant
4                                       American Burger
5                                     American Airlines
6                                    American Crew Shop
7                     American Express Customer Service
8                                   American Bbq Picniq
9         AC Seafood American And Vietnamese Streetfood
10                                                  W35
11                        American Express Magyarország
12                                    All American Food
13                                        American Rags
14                               AST - American Service
15                        American Burger & Steak House
16                                 All American Astoria
17                                        Americ

In [15]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### let's see how many restaurants found:

In [16]:
dataframe_filtered.shape

(33, 16)

In [17]:
mean_lat = dataframe_filtered['lat'].sum()/33
mean_lng = dataframe_filtered['lng'].sum()/33
print('The coordinates of the mean points of all the american restaurants and buffets are: '+str(mean_lat)+', '+str(mean_lng))

The coordinates of the mean points of all the american restaurants and buffets are: 47.50553237172409, 19.052409028352457


In [18]:
hotels_map = folium.Map(location=[mean_lat, mean_lng], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the mean
folium.features.CircleMarker(
    [mean_lat, mean_lng],
    radius=10,
    color='red',
    popup='mean',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hotels_map)

# display map
hotels_map

<h3>search for the closest hotel</h3>

In [19]:
mean_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "hotel",
    "near": "Budapest",
    "radius": 200,
    "limit": 100}

hotel_data = requests.get("https://api.foursquare.com/v2/venues/explore", params=mean_parameters)

In [20]:
h = hotel_data.json()["response"]
h.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [21]:
h["headerLocationGranularity"], h["headerLocation"], h["headerFullLocation"]

('city', 'Budapest', 'Budapest')

In [22]:
latitude = mean_lat
longitude = mean_lng
radius = 200
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [23]:
search_query = 'hotel'
url_h = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [24]:
results_h = requests.get(url_h).json()
results_h

{'meta': {'code': 200, 'requestId': '5f1b818de301787aa96a715d'},
 'response': {'venues': [{'id': '4bc77b2693bdeee1c76d37ae',
    'name': 'Hotel Parlament',
    'location': {'address': 'Kálmán Imre utca 19',
     'lat': 47.506880771077604,
     'lng': 19.05322590405708,
     'labeledLatLngs': [{'label': 'display',
       'lat': 47.506880771077604,
       'lng': 19.05322590405708}],
     'distance': 162,
     'postalCode': '1054',
     'cc': 'HU',
     'city': 'Budapest',
     'state': 'Budapest',
     'country': 'Magyarország',
     'formattedAddress': ['Budapest',
      'Kálmán Imre utca 19',
      '1054',
      'Magyarország']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1595638317',
    'hasPerk': False},
   {'id': '4bdf12c07ea362b5cb4e43c4',


In [25]:
# assign relevant part of JSON to venues
hotels = results_h['response']['venues']

# tranform venues into a dataframe
dataframe_h = json_normalize(hotels)
dataframe_h.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4bc77b2693bdeee1c76d37ae,Kálmán Imre utca 19,HU,Budapest,Magyarország,162,"[Budapest, Kálmán Imre utca 19, 1054, Magyaror...","[{'label': 'display', 'lat': 47.50688077107760...",47.506881,19.053226,1054,Budapest,Hotel Parlament,v-1595638317
1,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4bdf12c07ea362b5cb4e43c4,Hold Utca 3-5,HU,Budapest,Magyarország,196,"[Budapest, Hold Utca 3-5, 1054, Magyarország]","[{'label': 'display', 'lat': 47.50378456555193...",47.503785,19.052788,1054,Budapest,Hotel President Budapest,v-1595638317
2,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",False,5c5737f0acb00b002c817e7c,NaN,HU,Budapest V. kerület,Magyarország,175,"[Budapest V. kerület, 1054, Magyarország]","[{'label': 'display', 'lat': 47.504002, 'lng':...",47.504002,19.052992,1054,Boedapest,Hotel President | SPA & WELLNESS,v-1595638317
3,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,53f77d59498e9dac1e008b8d,NaN,HU,NaN,Magyarország,178,[Magyarország],"[{'label': 'display', 'lat': 47.50401327090036...",47.504013,19.053167,NaN,NaN,Heliport Panorama Terrace & Restaurant - Hotel...,v-1595638317
4,"[{'id': '4bf58dd8d48988d100941735', 'name': 'M...",False,50b477c2e4b0301d80a285d6,NaN,HU,NaN,Magyarország,198,[Magyarország],"[{'label': 'display', 'lat': 47.503769, 'lng':...",47.503769,19.052792,NaN,NaN,Hotel President Conference Room,v-1595638317


In [26]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe_h.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hotel Parlament,Hotel,Kálmán Imre utca 19,HU,Budapest,Magyarország,NaN,162,"[Budapest, Kálmán Imre utca 19, 1054, Magyaror...","[{'label': 'display', 'lat': 47.50688077107760...",47.506881,19.053226,NaN,1054,Budapest,4bc77b2693bdeee1c76d37ae
1,Hotel President Budapest,Hotel,Hold Utca 3-5,HU,Budapest,Magyarország,NaN,196,"[Budapest, Hold Utca 3-5, 1054, Magyarország]","[{'label': 'display', 'lat': 47.50378456555193...",47.503785,19.052788,NaN,1054,Budapest,4bdf12c07ea362b5cb4e43c4
2,Hotel President | SPA & WELLNESS,Spa,NaN,HU,Budapest V. kerület,Magyarország,NaN,175,"[Budapest V. kerület, 1054, Magyarország]","[{'label': 'display', 'lat': 47.504002, 'lng':...",47.504002,19.052992,NaN,1054,Boedapest,5c5737f0acb00b002c817e7c
3,Heliport Panorama Terrace & Restaurant - Hotel...,Restaurant,NaN,HU,NaN,Magyarország,NaN,178,[Magyarország],"[{'label': 'display', 'lat': 47.50401327090036...",47.504013,19.053167,NaN,NaN,NaN,53f77d59498e9dac1e008b8d
4,Hotel President Conference Room,Meeting Room,NaN,HU,NaN,Magyarország,NaN,198,[Magyarország],"[{'label': 'display', 'lat': 47.503769, 'lng':...",47.503769,19.052792,NaN,NaN,NaN,50b477c2e4b0301d80a285d6
5,Hotel President Breakfast Room,Hotel,Hold u.,HU,Budapest,Magyarország,NaN,198,"[Budapest, Hold u., Magyarország]","[{'label': 'display', 'lat': 47.50377258456925...",47.503773,19.052805,NaN,NaN,Budapest,5060074ce4b05548bddf0ca4
6,Hotel President Budapest Swimming Pool,Pool,NaN,HU,NaN,Magyarország,NaN,199,[Magyarország],"[{'label': 'display', 'lat': 47.50383541240558...",47.503835,19.053257,NaN,NaN,NaN,53e0bb39498e18848f6cf3e5
7,Roof Of President Hotel,Scenic Lookout,NaN,HU,NaN,Magyarország,NaN,188,[Magyarország],"[{'label': 'display', 'lat': 47.50389, 'lng': ...",47.503890,19.053043,NaN,NaN,NaN,50b76cade4b0492788e05291
8,wellness room at Parlament Hotel,Spa,NaN,HU,NaN,Magyarország,NaN,161,[Magyarország],"[{'label': 'display', 'lat': 47.506839, 'lng':...",47.506839,19.053327,NaN,NaN,NaN,56bf7a55cd10e9781c9185e4
9,Washington Room President Hotel,Meeting Room,NaN,HU,NaN,Magyarország,NaN,184,[Magyarország],"[{'label': 'display', 'lat': 47.503925, 'lng':...",47.503925,19.053026,NaN,NaN,NaN,50b4dad5e4b05f6f86b15760


In [27]:
winner = dataframe_filtered.sort_values(by='distance')
winner = winner.drop(columns=['crossStreet','neighborhood','id','city','country','postalCode','state','labeledLatLngs'])
winner = winner.dropna()
winner

,name,categories,address,cc,distance,formattedAddress,lat,lng
0,Hotel Parlament,Hotel,Kálmán Imre utca 19,HU,162,"[Budapest, Kálmán Imre utca 19, 1054, Magyaror...",47.506881,19.053226
1,Hotel President Budapest,Hotel,Hold Utca 3-5,HU,196,"[Budapest, Hold Utca 3-5, 1054, Magyarország]",47.503785,19.052788
5,Hotel President Breakfast Room,Hotel,Hold u.,HU,198,"[Budapest, Hold u., Magyarország]",47.503773,19.052805


In [28]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(winner.lat, winner.lng, winner.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [29]:
winner_hotel = pd.DataFrame(winner)
winner_hotel.iloc[0]

name                                                  Hotel Parlament
categories                                                      Hotel
address                                           Kálmán Imre utca 19
cc                                                                 HU
distance                                                          162
formattedAddress    [Budapest, Kálmán Imre utca 19, 1054, Magyaror...
lat                                                           47.5069
lng                                                           19.0532
Name: 0, dtype: object